In [1]:
import numpy as np
import pandas as pd

In [2]:
# def calc_info_gain(data,classes,feature):
#     gain = 0
#     nData = len(data)
#     # List the values that feature can take
#     values =[]
#     for datapoint in data:
#         if datapoint[feature] not in values:
#             values.append(datapoint[feature])
    
#     featureCounts = np.zeros(len(values))
#     entropy = np.zeros(len(values))
#     valueIndex = 0
#     #Find where those values appear in data[feature] and the corresponding class
#     for value in values:
#         dataIndex = 0
#         newClasses = []
#         for datapoint in data:
#             if datapoint[feature] == value:
#                 featureCounts[valueIndex]+=1
#                 newClasses.append(classes[dataIndex])
#             dataIndex +=1
#         # Get the values in newClasses
#         classValues = []
#         for aclass in newClasses:
#             if classValues.count(aclass)==0:
#                 classValues.append(aclass)
#         classCounts = np.zeros(len(classValues))
#         classIndex = 0
#         for classValue in classValues:
#             for aclass in newClasses:
#                 if aclass == classValue:
#                     classCounts[classIndex] +=1
#             classIndex +=1
            
#         for classIndex in range(len(classValues)):
#             entropy[valueIndex] += calc_entropy(float(classCounts[classIndex])/sum(classCounts))
#         gain += float(featureCounts[valueIndex])/nData*entropy[valueIndex]
#         valueIndex += 1
#     return gain


In [3]:
# def findPath(graph,start,end,pathSoFar):
#     pathSoFar = pathSoFar + [start]
#     if start == end:
#         return pathSoFar
#     if start not in graph:
#         return None
#     for node in graph[start]:
#         if node not in pathSoFar:
#             newpath = findPath(graph,node,end,pathSoFar)
#             return newpath
#     return None

In [4]:
# def make_tree(data,classes,featureNames):
#     #Various initialisations suppressed
#     default = classes[np.argmax(frequency)]
#     if nData ==0 or nFeatures ==0:
#         #Have reach empty branch
#         return default
#     elif classes.count(classes[0]) == nData:
#         #Only one class remain
#         return classes[0]
#     else:
#         #Choose which feature is best
#         gain = np.zeros(nFeatures)
#         for feature in range(nFeatures):
#             g = calc_info_gain(data,classes,feature)
#             gain[feature] = totalEntropy -g
#         bestFeature = np.argmax(gain)
#         tree = {featureNames[bestFeature]:{}}
#         #Find the possible feature values
#         for value in values:
#             # Find the datapoints with each feature value
#             for datapoint in data:
#                 if datapoint[bestFeature] == value:
#                     if bestFeature ==0:
#                         datapoint = datapoint[1:]
#                         newnames = featureNames[1:]
#                     elif bestFeature == nFeatures:
#                         datapoint = datapoint[:-1]
#                         newNames = featureNames[:-1]
#                     else:
#                         datapoint = datapoint[:bestFeature]
#                         datapoint.extend(datapoint[bestFeature+1:])
#                         newNames = featureNames[:bestFeature]
#                         newNames.extend(featureNames[bestFeature+1:])
#                     newData.append(datapoint)
#                     newClasses.append(classes[index])
#                 index+=1
#             #Now recurse to the next level
#             subtree = make_tree(newData,newClasses,newNames)
#             # And on returning, add the subtree on to the tree
#             tree[featureNames[bestFeature]][value] = subtree
#         return tree

In [5]:
class Node:
    def __init__(self,index,classes,label,depth=0,attribute=None,valueNames=None):
        self.feature = attribute
        self.values = valueNames
        self.index = index
        self.classes = classes
        self.valueNode = {}
        self.depth = depth
        
    def subnode(self, node,value):
        self.valueNode[value] = node


In [6]:
class Tree:
    
    def __init__(self,data,classes,label,max_depth):
        self.head = Node(data.index,classes,label)
        self.leaves = [self.head]
        self.max_depth = max_depth
        
    
    def branch(self,leaf,data,label,attr): 
        leaf.feature = attr
        leaf.values = set(data.iloc[leaf.index][leaf.feature])
        indexes = data.iloc[leaf.index].groupby([leaf.feature]).groups
        for name, index in indexes.items():
            classes = dict(data.iloc[index][label].value_counts())
            if len(classes) > 0:
                node = Node(index,classes,label,depth=leaf.depth+1)
                leaf.subnode(node,name)
                self.leaves.append(node)
        self.leaves.remove(leaf)
        
    
    def predict(self,data):
        predict = []
        for i in data.index:
  
            start = self.head
            while start.feature!=None:
                val = data.iloc[i][start.feature]
                if val in start.valueNode.keys():
                    start = start.valueNode[val]
                else:
                    break
            
            bClass = int()
            nClass = -1
            
            for i,j in start.classes.items():
                if j>nClass:
                    nClass = j
                    bClass = i
            predict.append(bClass)
        return np.array(predict)
    
    def calc_accuracy(self,predict,label):
        return (sum(label==predict)/len(label))
        

In [7]:
def calc_info_gain(data,label,feature):
    entropyS = data[[label]].apply(calc_entropy)
    featureCount = data.groupby([feature]).count().values[:,0]
    weights = featureCount/sum(featureCount)
    en = sum(data.groupby([feature])[label].apply(calc_entropy)* weights)
    gain = entropyS - en
    return gain[0]
    

In [8]:
def calc_entropy(data):
    values = set(data)
    count = {}
    for value in values:
        count[value] = 0
    for i in data:
        count[i]+=1
    counts = np.array(list(count.values()))
    p =list(counts/sum(counts))
    e = 0
    for i in p:
        if i!=0:
            e += -i * np.log2(i)     
    return e

In [9]:
def best_feature(dic):
    max_ = np.max(list(dic.values()))
    for key,value in dic.items():
        if value == max_:
            return key
    return None

In [10]:
def build_tree(data,features,label):
    classes = dict(data[label].value_counts())
    tree = Tree(data,classes,label,max_depth=6)
    position = 0
    
    while position <len(tree.leaves):

        leaf = tree.leaves[position]
        if (len(leaf.classes)<=1) or (len(data.iloc[leaf.index])<=1) or (leaf.depth >=tree.max_depth):
            position += 1
        else:
            dic_gain = {}
            for feature in features:
                info_gain = calc_info_gain(data[[feature,label]],label,feature)
                if (info_gain >5e-4) and (data.iloc[leaf.index][[feature]].apply(calc_entropy)[0]!=0):
                    dic_gain[feature] = info_gain
                    
            if len(dic_gain) >0:
                bFeature = best_feature(dic_gain)
                tree.branch(leaf,data,label,bFeature)
            else:
                position += 1
    return tree

In [11]:
# # Test tree
# a=Tree(df,['bad','good','normal'],'label')
# label = 'label'
# features = set(df.columns)-{'label'}
# dic_gain = {}
# for feature in features:
#     dic_gain[feature] = calc_info_gain(df[[feature,label]],label,feature)
# name = -1
# m = 0
# for i ,j in dic_gain.items():
#     if j>m:
#         m = j
#         name = i
# best_feature = name
# a.branch(tree.head,df,label,best_feature)

In [12]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv('data/test.csv')

In [13]:
train_df = train_df.drop(['Cabin','PassengerId','Name','Fare','Ticket'],axis=1)

train_df[['Embarked']] = train_df[['Embarked']].fillna(train_df['Embarked'].mode()[0])
train_df[['Age']] = train_df[['Age']].fillna(train_df['Age'].mean())
test_df[['Embarked']] = test_df[['Embarked']].fillna(train_df['Embarked'].mode()[0])

test_df[['Age']] = test_df[['Age']].fillna(train_df['Age'].mean())
test_df = test_df.drop(['Cabin','PassengerId','Name','Fare','Ticket'],axis=1)


train_df[['Parch']]= pd.cut(train_df['Parch'],3,labels=[1,2,3])
train_df[['Age']]= pd.cut(train_df['Age'],4,labels=[1,2,3,4])
train_df[['SibSp']] = pd.cut(train_df['SibSp'],3,labels=[1,2,3])


test_df[['Parch']]= pd.cut(test_df['Parch'],3,labels=[1,2,3])
test_df[['Age']]= pd.cut(test_df['Age'],4,labels=[1,2,3,4])
test_df[['SibSp']] = pd.cut(test_df['SibSp'],3,labels=[1,2,3])

In [14]:
# def categorical_age(x):
#     if x<16:
#         return 'Child'
#     elif x<40:
#         return 'young'
#     elif x<60:
#         return 'growup'
#     else:
#         return 'old'

# train_df[['Age']] = train_df[['Age']].applymap(categorical_age)
# test_df[['Age']] = test_df[['Age']].fillna(test_df.mean())
# test_df[['Age']] = test_df[['Age']].applymap(categorical)

In [15]:
tree = build_tree(train_df,set(train_df.columns)-{'Survived'},'Survived')
tree.calc_accuracy(tree.predict(train_df),train_df['Survived'])

0.8181818181818182

In [16]:
sub = pd.read_csv("data/gender_submission.csv")
sub = sub.drop(['PassengerId'],axis=1)

In [17]:
data = {k:[v] for k,v in dict(test_df.iloc[48]).items()}
dataframe = pd.DataFrame(data = data)
# for i in dataframe.index:
#     print(i)
#     print(dataframe.iloc[i])
tree.predict(dataframe)

array([1])

In [18]:
tree.calc_accuracy(tree.predict(test_df),sub['Survived'])

0.8899521531100478

In [19]:
def visit(node):
    parent=node
    

In [20]:
for i in tree.leaves:
    if len(i.classes)==0:
        print(i)

In [21]:
def visit(x):
    if x.feature==None or len(x.valueNode)==0:
        return
    else:
        print(x.feature)
        for i in x.valueNode.values():
            visit(i)
        
        

In [22]:
for i in tree.leaves:
    if len(i.classes)==0 and i.depth==1:
        print(i)

In [23]:
from graphviz import Digraph

In [24]:
dot = Digraph(comment='The Round Table')

In [25]:
dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')
dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')